In [31]:
import pandas as pd
from WorkforceSentimentMonitoring.data import get_data


/Users/theresasporn/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


IndentationError: expected an indented block (data.py, line 154)

In [28]:
from
submission, train, test = get_data("/Users/theresasporn/code/theresa-sporn/workforce_sentiment_monitoring/raw_data")

/Users/theresasporn/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'get_data' is not defined

In [25]:
text = merge(submission, train, test)
#positives = text['positives']
#positives.head(5)
text.head()

/Users/theresasporn/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'merge' is not defined

In [26]:
text[1]

/Users/theresasporn/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'text' is not defined

In [4]:
df = pd.read_csv("/Users/theresasporn/Downloads/pos_neg_df.csv")
df.head()

,positives,negatives
0,work people passionate care greater good fanta...,need open office many office sunlight
1,lot knowledge flexible hour office process rea...,key problem company lost true leader existing ...
2,immense carrier opportunity growth opportunity...,working fc first difficult week get habitual s...
3,employee food benefit perk shuttle basically e...,company become way vertical performance evalua...
4,good pay somewhat nice people,hired ’ always something


In [29]:
from WorkforceSentimentMonitoring.data import get_prepaired_data

/Users/theresasporn/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


IndentationError: expected an indented block (data.py, line 154)

In [5]:
type(df.negatives[0])

str

In [6]:
def extract_negative(df):
    """return df with negative reviews and their labels"""
    negatives = df[['negatives']]
    #negatives.loc[:,'sentiment'] = 0 # 0=> negative
    return negatives

In [7]:
negatives = extract_negative(df)
negatives.head(5)

,negatives
0,need open office many office sunlight
1,key problem company lost true leader existing ...
2,working fc first difficult week get habitual s...
3,company become way vertical performance evalua...
4,hired ’ always something


In [8]:
from WorkforceSentimentMonitoring.preprocessing import tokenize

In [9]:
negatives_tokenized = df['negatives'].apply(tokenize)
negatives_tokenized

0             [need, open, office, many, office, sunlight]
1        [key, problem, company, lost, true, leader, ex...
2        [working, fc, first, difficult, week, get, hab...
3        [company, become, way, vertical, performance, ...
4                            [hired, ’, always, something]
5        [pay, low, opportunity, move, within, pay, sca...
6        [lot, lot, meeting, top, decision, making, lot...
7        [environment, hyper, competitive, everybody, r...
8        [management, matured, always, end, micro, mana...
9                                           [amazon, scam]
10       [difficult, work, environment, useless, compet...
11                        [office, politics, slow, career]
12       [discrimination, back, stabbing, retaliation, ...
13       [working, ar, site, suck, oak, micromanaged, t...
14       [job, role, limited, hard, get, promotion, sal...
15       [hardly, decency, mutual, respect, among, empl...
16       [amount, work, rapid, changing, environment, p.

In [28]:
def extract_positive(df):
    """return df with positive reviews and their labels"""
    positives = df[['positives']]
    #text.loc[:,'sentiment'] = 1 # 1=> positive
    return positives

In [29]:
positives = extract_positive(df)
positives.head(5)

,positives
0,work people passionate care greater good fanta...
1,lot knowledge flexible hour office process rea...
2,immense carrier opportunity growth opportunity...
3,employee food benefit perk shuttle basically e...
4,good pay somewhat nice people


In [19]:
import gensim
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
from gensim.corpora import Dictionary
import pyLDAvis
import pyLDAvis.gensim

id2word = corpora.Dictionary(negatives_tokenized)
dictionary = id2word
dictionary = dictionary.filter_extremes(no_above=0.75)

texts = negatives_tokenized
corpus = [id2word.doc2bow(text) for text in texts]

ldamallet = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=2, id2word=id2word, iterations=100)

coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()

    
def compute_coherence_values(dictionary, corpus, texts, limit=8, start=2, step=1):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=id2word, alpha=.91)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=4, limit=8, step=2)


max_y = max(coherence_values)  # Find the maximum y value
max_x = coherence_values.index(max(coherence_values))  # Find the x value corresponding to the maximum y value
#xmax = x[numpy.argmax(y)]

optimal_model = model_list[coherence_values.index(max(coherence_values))]

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(optimal_model, corpus, id2word)
vis


/Users/theresasporn/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/theresasporn/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.169735  0.015397       1        1  20.918993
4     -0.026156  0.096361       2        1  18.695835
1      0.003273 -0.013472       3        1  16.809866
2     -0.146157 -0.003407       4        1  15.707266
5     -0.009394 -0.015386       5        1  14.631107
0      0.008698 -0.079493       6        1  13.236930, topic_info=     Category          Freq         Term         Total  loglift  logprob
206   Default   4216.000000         hour   4216.000000  30.0000  30.0000
169   Default   4757.000000         team   4757.000000  29.0000  29.0000
265   Default   3109.000000          day   3109.000000  28.0000  28.0000
338   Default   6793.000000         time   6793.000000  27.0000  27.0000
22    Default   6980.000000      company   6980.000000  26.0000  26.0000
171   Default  13027.000000         work  13027.000000  25.0000  25.0000
172   Default   1650.000000     politics   1650.000000  24.0000  24.0000
197   Default   1191.000000        break   1191.000000  23.0000  23.0000
129   Default   2280.000000          pay   2280.000000  22.0000  22.0000
219   Default   1478.000000        shift   1478.000000  21.0000  21.0000
93    Default   2078.000000        level   2078.000000  20.0000  20.0000
145   Default   4974.000000     employee   4974.000000  19.0000  19.0000
121   Default   1418.000000            ’   1418.000000  18.0000  18.0000
1230  Default   1269.000000    microsoft   1269.000000  17.0000  17.0000
138   Default   3892.000000          lot   3892.000000  16.0000  16.0000
241   Default   3535.000000         life   3535.000000  15.0000  15.0000
236   Default    983.000000       salary    983.000000  14.0000  14.0000
924   Default   1685.000000        apple   1685.000000  13.0000  13.0000
232   Default   2804.000000         hard   2804.000000  12.0000  12.0000
1010  Default   1450.000000        still   1450.000000  11.0000  11.0000
934   Default    934.000000       google    934.000000  10.0000  10.0000
238   Default   2745.000000      balance   2745.000000   9.0000   9.0000
136   Default   1690.000000       change   1690.000000   8.0000   8.0000
210   Default    710.000000       minute    710.000000   7.0000   7.0000
217   Default   1004.000000         rate   1004.000000   6.0000   6.0000
421   Default   5786.000000      manager   5786.000000   5.0000   5.0000
103   Default   2032.000000    sometimes   2032.000000   4.0000   4.0000
204   Default   1966.000000         high   1966.000000   3.0000   3.0000
689   Default   3623.000000         like   3623.000000   2.0000   2.0000
166   Default   2779.000000       amazon   2779.000000   1.0000   1.0000
...       ...           ...          ...           ...      ...      ...
204    Topic6    833.910950         high   1966.217285   1.1644  -4.6051
509    Topic6    558.497009       better   1236.931885   1.2270  -5.0060
2098   Topic6    127.908722     deadline    196.209717   1.5943  -6.4799
1      Topic6    880.299377         need   2285.727051   1.0680  -4.5510
444    Topic6    362.408875     pressure    772.742432   1.2650  -5.4385
138    Topic6   1198.108276          lot   3892.728027   0.8438  -4.2428
338    Topic6   1738.109985         time   6793.372559   0.6590  -3.8707
238    Topic6    870.727966      balance   2745.131348   0.8739  -4.5619
421    Topic6   1466.788208      manager   5786.486816   0.6497  -4.0404
128    Topic6    428.948944          low   1099.297607   1.0811  -5.2699
241    Topic6    944.144165         life   3535.916016   0.7017  -4.4810
171    Topic6   2309.818848         work  13027.294922   0.2923  -3.5863
506    Topic6    874.931213         much   3184.106445   0.7304  -4.5571
169    Topic6   1132.810425         team   4757.310547   0.5872  -4.2988
148    Topic6    834.318726         good   3016.481445   0.7369  -4.6046
391    Topic6    322.920502     engineer    752.692139   1.1759  -5.5538
146    Topic6    501.

In [ ]:
vis = pyLDAvis.gensim.prepare(ldamallet, corpus, id2word)
pyLDAvis.save_html(vis, 'pyLDAvis_negatives_tokenized.html')
#Where 'vis' is the name of your model, like this:


/Users/theresasporn/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
df = pd.read_csv("/Users/theresasporn/code/theresa-sporn/workforce_sentiment_monitoring/raw_data/train.csv")
df

/Users/theresasporn/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,ID,Place,location,date,status,job_title,summary,positives,negatives,advice_to_mgmt,score_1,score_2,score_3,score_4,score_5,score_6,overall
0,1,startup_1,NaN,"Dec 11, 2018",Current Employee,Anonymous Employee,Best Company to work for,People are smart and friendly,Bureaucracy is slowing things down,NaN,4.0,5.0,5.0,4.0,5.0,0,5.0
1,2,startup_1,"Mountain View, CA","Jun 21, 2013",Former Employee,Program Manager,"Moving at the speed of light, burn out is inev...","1) Food, food, food. 15+ cafes on main campus ...",1) Work/life balance. What balance? All those ...,1) Don't dismiss emotional intelligence and ad...,2.0,3.0,3.0,5.0,3.0,2094,5.0
2,3,startup_1,"New York, NY","May 10, 2014",Current Employee,Software Engineer III,Great balance between big-company security and...,"* If you're a software engineer, you're among ...","* It *is* becoming larger, and with it comes g...",Keep the focus on the user. Everything else wi...,5.0,4.0,5.0,5.0,4.0,949,5.0
3,4,startup_1,"Mountain View, CA","Feb 8, 2015",Current Employee,Anonymous Employee,The best place I've worked and also the most d...,You can't find a more well-regarded company th...,I live in SF so the commute can take between 1...,Keep on NOT micromanaging - that is a huge ben...,2.0,5.0,5.0,4.0,5.0,498,4.0
4,10,startup_1,NaN,"Dec 9, 2018",Current Employee,Anonymous Employee,Execellent for engineers,Impact driven. Best tech in the world.,Size matters. Engineers are a bit disconnected...,NaN,5.0,5.0,5.0,5.0,5.0,0,4.0
5,16,startup_1,NaN,"Dec 6, 2018",Current Employee,Anonymous Employee,Review,The people are great to work with,There is lower compensation than expected,NaN,NaN,NaN,NaN,NaN,NaN,0,5.0
6,17,startup_1,NaN,"Nov 29, 2018",Current Employee,Anonymous Employee,Review of Google,"The people are great to work with, good perks.",Wide variety in experience based on team.,NaN,4.0,5.0,5.0,5.0,4.0,2,3.0
7,18,startup_1,NaN,"Dec 2, 2018",Former Employee,Anonymous Employee,Analysts,"Good managers, benefits , some support, nice a...","Long hours, far locations, a lot of work",NaN,NaN,NaN,2.0,NaN,NaN,1,5.0
8,19,startup_1,NaN,"Dec 1, 2018",Former Employee,Anonymous Employee,Great Company With No Down Sides,Really fun work environment with startup,Really no down sides that I saw.,NaN,5.0,5.0,5.0,5.0,5.0,1,5.0
9,24,startup_1,NaN,"Nov 30, 2018",Current Employee,Anonymous Employee,Software Engineer,Awesome job environment to work in,Pressure is there for some parts,NaN,NaN,NaN,NaN,NaN,NaN,0,5.0


In [37]:
df = df[['negatives', 'score_1', "score_2", 'score_3', "score_4", "score_5", "overall"]]
df

/Users/theresasporn/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,negatives,score_1,score_2,score_3,score_4,score_5,overall
0,Bureaucracy is slowing things down,4.0,5.0,5.0,4.0,5.0,5.0
1,1) Work/life balance. What balance? All those ...,2.0,3.0,3.0,5.0,3.0,5.0
2,"* It *is* becoming larger, and with it comes g...",5.0,4.0,5.0,5.0,4.0,5.0
3,I live in SF so the commute can take between 1...,2.0,5.0,5.0,4.0,5.0,4.0
4,Size matters. Engineers are a bit disconnected...,5.0,5.0,5.0,5.0,5.0,4.0
5,There is lower compensation than expected,NaN,NaN,NaN,NaN,NaN,5.0
6,Wide variety in experience based on team.,4.0,5.0,5.0,5.0,4.0,3.0
7,"Long hours, far locations, a lot of work",NaN,NaN,2.0,NaN,NaN,5.0
8,Really no down sides that I saw.,5.0,5.0,5.0,5.0,5.0,5.0
9,Pressure is there for some parts,NaN,NaN,NaN,NaN,NaN,5.0


In [42]:
df_pos_work_life_balance = df['negatives'] if df.score_1 > 3
df_pos_work_life_balance 

/Users/theresasporn/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


SyntaxError: invalid syntax (<ipython-input-42-2cc04df257b6>, line 1)